In [1]:
import pandas as pd
import os
import numpy as np

C:\Users\Cafral\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Cafral\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Cafral\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#reading in the data
DIR='C:/Users/Cafral/Desktop/ICLR/favorita-grocery-sales-forecasting/data'

In [3]:
os.listdir(DIR)

['holidays_events.csv',
 'items.csv',
 'oil.csv',
 'sample_submission.csv',
 'stores.csv',
 'test.csv',
 'train.csv',
 'transactions.csv']

In [4]:
#loading the data
training = pd.read_csv(os.path.join(DIR,'train.csv'))
test = pd.read_csv(os.path.join(DIR,'test.csv'))
holidays_events = pd.read_csv(os.path.join(DIR,'holidays_events.csv'))
oil = pd.read_csv(os.path.join(DIR,'oil.csv'))
stores = pd.read_csv(os.path.join(DIR,'stores.csv'))
transactions = pd.read_csv(os.path.join(DIR,'transactions.csv'))
items =pd.read_csv(os.path.join(DIR,'items.csv'))

C:\Users\Cafral\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
items.columns

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')

In [5]:
training['date'] = pd.to_datetime(training['date'] )
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
oil['date'] = pd.to_datetime(oil['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

In [16]:
_ids_to_keep = ~training[['store_nbr', 'item_nbr']].isna()
_ids_to_keep.sum()

store_nbr    125497040
item_nbr     125497040
dtype: int64

In [42]:
training.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object')

In [17]:
training.drop_duplicates(inplace=True)
training.shape

(125497040, 6)

In [27]:
#figuring out the embeddings
embeddings_df = training.merge(holidays_events,on=['date'])
embeddings_df = embeddings_df.merge(stores,on=['store_nbr'])
embeddings_df = embeddings_df.merge(items,on=['item_nbr'])
embeddings_df.rename(columns={'type_x':'holiday_type','type_y':'store_type'},inplace=True)
embeddings_df.shape

group_by_cols = ['holiday_type',
                 'locale',
                 'locale_name',
                 'description',
                 'transferred',
                 'city',
                 'state',
                 'store_type',
                 'cluster',
                 'family',
                 'class',
                 'perishable']

group_ids = embeddings_df.groupby(by=group_by_cols,as_index=False).ngroup()
embeddings_df['metadata_id'] = group_ids

(21828208, 18)

In [38]:
group_ids.nunique()

1010343

In [44]:
#assigning embeddings id to non-static training data
training = training.merge(embeddings_df[['date','store_nbr','item_nbr','metadata_id']],on = ['date','store_nbr','item_nbr'])
training.shape

(21828208, 7)

In [45]:
# they use all the data so merging : how to incorporate meta data?
training = training.merge(oil,on=['date'])
print(training.shape)
training = training.merge(transactions,on=['date', 'store_nbr'])
print(training.shape)

#training = training.merge(holidays_events,on=['date'])
#print(training.shape)
#training = training.merge(stores,on=['store_nbr'])
#print(training.shape)

(15204171, 8)
(15202453, 9)


In [46]:
# processing the data : for details refer to APPENDIX of https://arxiv.org/pdf/1912.09363.pdf

# 1. We treat each product number-store number pair as a separate entity, with over 135k entities in total. 
training.shape

(15202453, 9)

In [47]:
# The training set is made up of 450k samples taken between 2015-01-01 to 2015-12-01
train_period_mask = ((training['date']>=pd.to_datetime('2015-01-01')) & (training['date']<=pd.to_datetime('2015-12-01')))
training_paper = training[train_period_mask]

# validation set of 50k samples from the 30 days after the training set,
end_val_period = training['date'][(training['date']>pd.to_datetime('2015-12-01'))].unique()[30]
val_period_mask = ((training['date']>pd.to_datetime('2015-12-01')) & \
                   (training['date']<=end_val_period))
validation_paper = training[val_period_mask]

# test set of all entities over the 30-day horizon following the validation set.
end_test_period = training['date'][(training['date']>end_val_period)].unique()[30]
test_period_mask = (((training['date']>end_val_period)) & (training['date']<=end_test_period))
test_paper = training[test_period_mask]

In [50]:
training_paper.shape #doesn't add up with nos in paper

(2487480, 9)

In [51]:
validation_paper.shape  #doesn't add up with nos in paper

(3121416, 9)

In [52]:
test_paper.shape  #doesn't add up with nos in paper

(3251181, 9)

In [56]:
training_paper[['store_nbr', 'item_nbr']].drop_duplicates().shape #around the no. mentioned in the paper

(133656, 2)

In [57]:
validation_paper[['store_nbr', 'item_nbr']].drop_duplicates().shape  #around the no. mentioned in the paper

(139223, 2)

In [58]:
test_paper[['store_nbr', 'item_nbr']].drop_duplicates().shape 

(155702, 2)

In [59]:
# We include an additional ’open’ flag to denote whether data is present on a given day.
training_paper['open'] = training_paper.isna().apply(lambda x: all(x), axis=1)
validation_paper['open'] = validation_paper.isna().apply(lambda x: all(x), axis=1)
test_paper['open'] = test_paper.isna().apply(lambda x: all(x), axis=1)

C:\Users\Cafral\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cafral\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Cafral\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [60]:
# Data is resampled at regular daily intervals (????????????????????????),
#imputing any missing days using the last available observation
training_paper = training_paper.ffill()
validation_paper = validation_paper.ffill()
test_paper = test_paper.ffill()

In [61]:
# We apply a log-transform on the sales data, and adopt z-score normalization across all entities
training_paper['unit_sales'] = np.log(training_paper['unit_sales'])
validation_paper['unit_sales'] = np.log(validation_paper['unit_sales'])
test_paper['unit_sales'] =np.log(test_paper['unit_sales'])

C:\Users\Cafral\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [62]:
training_paper

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,metadata_id,dcoilwtico,transactions,open
4352619,38594264,2015-01-01,25,103665,2.484907,False,865940,NaN,2202,False
4352620,38594265,2015-01-01,25,105575,3.135494,False,866042,NaN,2202,False
4352621,38594266,2015-01-01,25,108634,0.000000,False,866057,NaN,2202,False
4352622,38594267,2015-01-01,25,108698,1.791759,False,865996,NaN,2202,False
4352623,38594268,2015-01-01,25,108786,1.791759,False,865967,NaN,2202,False
...,...,...,...,...,...,...,...,...,...,...
6840094,63590977,2015-11-30,54,2026650,0.693147,False,142009,40.43,835,False
6840095,63590978,2015-11-30,54,2026801,1.609438,False,142024,40.43,835,False
6840096,63590979,2015-11-30,54,2026858,0.000000,False,142009,40.43,835,False
6840097,63590980,2015-11-30,54,2026983,1.386294,False,142009,40.43,835,False


In [64]:
from sklearn.preprocessing import LabelEncoder 

In [66]:
#  We consider log sales, transactions, oil to be real-valued and the rest to be categorical - does encoding things matter in this case?

#The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr.
le_onpromotion = LabelEncoder()
le_open = LabelEncoder()
training_paper['onpromotion'] = le_onpromotion.fit_transform(training_paper['onpromotion'].values)
training_paper['open'] = le_open.fit_transform(training_paper['open'].values)
validation_paper['onpromotion'] = le_onpromotion.transform(validation_paper['onpromotion'].values)
validation_paper['open'] = le_open.transform(validation_paper['open'].values)
test_paper['onpromotion'] = le_onpromotion.transform(test_paper['onpromotion'].values)
test_paper['open'] = le_open.transform(test_paper['open'].values)

# QESTION : should we encode the embedings too?

# to do encode these or not?
# training_paper['item_nbr'] = le.fit_transform(training_paper['item_nbr'].values)
# training_paper['store_nbr'] = le.fit_transform(training_paper['store_nbr'].values)

In [67]:
SAVE_DIR='C:/Users/Cafral/Desktop/ICLR/favorita-grocery-sales-forecasting/final_dfs'
training_paper.to_csv('training_data.csv',index=False)
validation_paper.to_csv('validation_paper.csv',index=False)
test_paper.to_csv('test_paper.csv',index=False)

In [ ]:
# adopt z-score normalization across all entities - to be done in flow forecast pipeline : std scaler